# PyCoMo Maximum Growth-Rate #
This tutorial show-cases how to find the maximum growth-rate of a community - across all possible abundance profiles.

The expected runtime for this notebook is less than 10 minutes.

In [1]:
from pathlib import Path
import sys
import os
import cobra
import math 
import time
import warnings

In [2]:
path_root = "../src"  # Change path according to your PyCoMo location
sys.path.append(str(path_root))
import pycomo
pycomo.configure_logger(level="info")

2025-02-11 16:57:51,049 - pycomo - INFO - Logger initialized.
2025-02-11 16:57:51,051 - pycomo - INFO - Process Pool Logger initialized.
2025-02-11 16:57:51,051 - pycomo - INFO - Utils Logger initialized.
2025-02-11 16:57:51,053 - pycomo - INFO - Multiprocess Logger initialized.
2025-02-11 16:57:51,054 - pycomo - INFO - Log level set to info


## Load the Toy Models ##
These are simple toy models of 2 organisms. In a community, they are expexted to grow at a maximum of 10/h, however one organism alone can grow as fast as 15/h. Both cases can be calculated.

In [3]:
# create two cobra models from sbml (change the path according to where the models are located)
toy1 = cobra.io.read_sbml_model("../data/use_case/toy_models/toy_1.xml")
toy2 = cobra.io.read_sbml_model("../data/use_case/toy_models/toy_2_2.xml")

# create Single Organism Models from cobra models
Toy1 = pycomo.SingleOrganismModel(toy1, "toy1")
Toy2 = pycomo.SingleOrganismModel(toy2, "toy2")

# create Community Model from Single Organism Models
C = pycomo.CommunityModel([Toy1, Toy2], name = "Toy_community")
# instantiate the model and set cplex as a solver (if available)
#C.model.solver = "cplex"

## Compute maximum community growth rate

The method ```max_growth_rate``` calculates the overall maximum growth rate of the community, regardless of its composition.

In [4]:
C.max_growth_rate()

2025-02-11 16:57:51,118 - pycomo - INFO - No community model generated yet. Generating now:
2025-02-11 16:57:51,126 - pycomo - INFO - Identified biomass reaction from objective: R4
2025-02-11 16:57:51,146 - pycomo - INFO - Identified biomass reaction from objective: R4
2025-02-11 16:57:51,207 - pycomo - WARNING - Not all reactions in the model are mass and charge balanced. To check which reactions are imbalanced, please run the get_unbalanced_reactions method of this CommunityModel object
2025-02-11 16:57:51,209 - pycomo - INFO - Generated community model.
2025-02-11 16:57:51,212 - pycomo - INFO - New round: lb: 0.0, ub: 1000.0, x: 2e-06
2025-02-11 16:57:52,205 - pycomo - INFO - Logger initialized.
2025-02-11 16:57:52,206 - pycomo - INFO - Process Pool Logger initialized.
2025-02-11 16:57:52,206 - pycomo - INFO - Utils Logger initialized.
2025-02-11 16:57:52,208 - pycomo - INFO - Multiprocess Logger initialized.
2025-02-11 16:57:52,217 - pycomo - INFO - Logger initialized.
2025-02-11 1

np.float64(15.0)

```max_growth_rate``` has the following parameters:
* ```minimal_abundance``` (default = 0)
* ```return_abundances``` (default = False)
* ```sensitivity``` (default = 6)
* ```gurobi``` (default = False)

### Return Abundances
By setting the parameter ```return_abundances``` to ```True```, additional information about the feasible community compositions at the maximum growth rate is returned in the form of a pandas dataframe.

The minimal flux of x_fraction_reaction corresponds to the minimal abundance of member x in the community. Likewise, the maximal flux corresponds to the maximal abundance of member x.

In [5]:
C.max_growth_rate(return_abundances=True)

2025-02-11 16:57:55,317 - pycomo - INFO - New round: lb: 0.0, ub: 1000.0, x: 2e-06
2025-02-11 16:57:56,280 - pycomo - INFO - Logger initialized.
2025-02-11 16:57:56,280 - pycomo - INFO - Process Pool Logger initialized.
2025-02-11 16:57:56,280 - pycomo - INFO - Utils Logger initialized.
2025-02-11 16:57:56,282 - pycomo - INFO - Multiprocess Logger initialized.
2025-02-11 16:57:56,297 - pycomo - INFO - Logger initialized.
2025-02-11 16:57:56,298 - pycomo - INFO - Process Pool Logger initialized.
2025-02-11 16:57:56,298 - pycomo - INFO - Utils Logger initialized.
2025-02-11 16:57:56,299 - pycomo - INFO - Multiprocess Logger initialized.
2025-02-11 16:57:56,312 - pycomo - INFO - Log level set to 20
2025-02-11 16:57:56,315 - pycomo - INFO - Processed 100.0% of fva steps
2025-02-11 16:57:56,333 - pycomo - INFO - New round: lb: 10.0, ub: 1000.0, x: 10.000002
2025-02-11 16:57:57,234 - pycomo - INFO - Logger initialized.
2025-02-11 16:57:57,234 - pycomo - INFO - Process Pool Logger initialized

reaction_id  min_flux  max_flux
0  toy1_fraction_reaction       0.0       0.0
1  toy2_fraction_reaction       1.0       1.0
2       community_biomass      15.0      15.0

### Minimal abundance
Edge cases exist in which the highest growth rate of a community is not achieved by a combination of all the members. Instead, one member with an abundance of 100% may achieve the maximum growth rate. 

In order to exclude these cases, the parameter ```minimal_abundance``` must be set to a float greater than 0. This sets the minimal abundance of all members to the set float. Take care that the sum of the minimal abundances is not greater than 1, as this will result in an error.

In [6]:
C.max_growth_rate(minimal_abundance=0.1)

2025-02-11 16:58:00,355 - pycomo - INFO - New round: lb: 0.0, ub: 1000.0, x: 2e-06
2025-02-11 16:58:01,304 - pycomo - INFO - Logger initialized.
2025-02-11 16:58:01,305 - pycomo - INFO - Process Pool Logger initialized.
2025-02-11 16:58:01,305 - pycomo - INFO - Utils Logger initialized.
2025-02-11 16:58:01,307 - pycomo - INFO - Multiprocess Logger initialized.
2025-02-11 16:58:01,312 - pycomo - INFO - Logger initialized.
2025-02-11 16:58:01,312 - pycomo - INFO - Process Pool Logger initialized.
2025-02-11 16:58:01,313 - pycomo - INFO - Utils Logger initialized.
2025-02-11 16:58:01,314 - pycomo - INFO - Multiprocess Logger initialized.
2025-02-11 16:58:01,335 - pycomo - INFO - Log level set to 20
2025-02-11 16:58:01,340 - pycomo - INFO - Processed 100.0% of fva steps
2025-02-11 16:58:01,357 - pycomo - INFO - New round: lb: 10.0, ub: 1000.0, x: 10.000002
2025-02-11 16:58:02,281 - pycomo - INFO - Logger initialized.
2025-02-11 16:58:02,281 - pycomo - INFO - Process Pool Logger initialized

np.float64(10.0)

In [7]:
# too large minimal_abundance value leads to an error
try:
    C.max_growth_rate(minimal_abundance=0.6)
except ValueError:
    print("Error thrown!")
    
# The community has two members: 
# 2 * 0.6 = 1.2
# 1.2 > 1

Error thrown!


## Example: Biogas producing community ##
The example for this part will be a three member community published by Koch et al. 2019 (https://doi.org/10.1371/journal.pcbi.1006759). The three member organisms are representatives of functional guilds in a biogas community.

In [8]:
# create cobra models from sbml
dv = cobra.io.read_sbml_model("../data/use_case/koch/dv.xml")
mh = cobra.io.read_sbml_model("../data/use_case/koch/mh.xml")
mb = cobra.io.read_sbml_model("../data/use_case/koch/mb.xml")

# change infinite upper bounds to 1000
for model in [dv,mh,mb]:
    for reaction in model.reactions:
        if reaction.upper_bound == math.inf:
            reaction.upper_bound = 1000.
        if reaction.lower_bound == -math.inf:
            reaction.lower_bound = -1000.

# create Single Organism Models from cobra models
DV = pycomo.SingleOrganismModel(dv, "dv")
MH = pycomo.SingleOrganismModel(mh, "mh")
MB = pycomo.SingleOrganismModel(mb, "mb")

# create Community Model from Single Organism Models
C2 = pycomo.CommunityModel([DV, MH, MB], name = "dv_mh_mb_community")

# set solver to cplex (if available)
#C2.model.solver = "cplex"

# apply the same medium as in the paper by Koch et al.
medium = {
    'EX_CO2_EX_medium': 1000.0,
    'EX_Eth_EX_medium': 1000.0,
    'EX_BM_tot_medium': 1000.0
}
C2.medium = medium
C2.apply_medium()

# Formate and Hydrogen are not allowed to accumulate in the medium.
C2.model.reactions.get_by_id("EX_Form_EX_medium").upper_bound = 0.
C2.model.reactions.get_by_id("EX_H2_EX_medium").upper_bound = 0.


'3PG' is not a valid SBML 'SId'.
'2PG' is not a valid SBML 'SId'.
'2PG__PEP' is not a valid SBML 'SId'.
'3PG__2PG' is not a valid SBML 'SId'.
'0Pyr__AcCoA' is not a valid SBML 'SId'.
'5CHOMPT' is not a valid SBML 'SId'.
'3PG' is not a valid SBML 'SId'.
'2PG' is not a valid SBML 'SId'.
'2PG__3PG' is not a valid SBML 'SId'.
'3PG__DPG' is not a valid SBML 'SId'.
'5CHOMPT__CHH4MPT' is not a valid SBML 'SId'.
'3PG' is not a valid SBML 'SId'.
'2PG' is not a valid SBML 'SId'.
'5CHOMPT' is not a valid SBML 'SId'.
'3PG__2PG__3PG' is not a valid SBML 'SId'.
'5CHOMPT__CHH4MPT' is not a valid SBML 'SId'.
2025-02-11 16:58:04,460 - pycomo - INFO - No community model generated yet. Generating now:
2025-02-11 16:58:04,473 - pycomo - INFO - Identified biomass reaction from objective: r_BMDV2BMc
2025-02-11 16:58:04,607 - pycomo - INFO - Identified biomass reaction from objective: BM_Synth
2025-02-11 16:58:04,798 - pycomo - INFO - Identified biomass reaction from objective: BM_Synth
2025-02-11 16:58:05,0

In [9]:
# calculate max growth rate of community
C2.max_growth_rate()

2025-02-11 16:58:05,131 - pycomo - INFO - New round: lb: 0.0, ub: 1000.0, x: 2e-06
2025-02-11 16:58:06,131 - pycomo - INFO - Logger initialized.
2025-02-11 16:58:06,131 - pycomo - INFO - Process Pool Logger initialized.
2025-02-11 16:58:06,131 - pycomo - INFO - Utils Logger initialized.
2025-02-11 16:58:06,133 - pycomo - INFO - Multiprocess Logger initialized.
2025-02-11 16:58:06,234 - pycomo - INFO - Log level set to 20
2025-02-11 16:58:06,999 - pycomo - INFO - Logger initialized.
2025-02-11 16:58:07,000 - pycomo - INFO - Process Pool Logger initialized.
2025-02-11 16:58:07,000 - pycomo - INFO - Utils Logger initialized.
2025-02-11 16:58:07,001 - pycomo - INFO - Multiprocess Logger initialized.
2025-02-11 16:58:07,100 - pycomo - INFO - Log level set to 20
2025-02-11 16:58:08,108 - pycomo - INFO - Logger initialized.
2025-02-11 16:58:08,109 - pycomo - INFO - Process Pool Logger initialized.
2025-02-11 16:58:08,109 - pycomo - INFO - Utils Logger initialized.
2025-02-11 16:58:08,111 - py

np.float64(0.052128)

In [10]:
# calculate abundance profile as well
C2.max_growth_rate(return_abundances=True)

2025-02-11 16:58:11,085 - pycomo - INFO - New round: lb: 0.0, ub: 1000.0, x: 2e-06
2025-02-11 16:58:12,016 - pycomo - INFO - Logger initialized.
2025-02-11 16:58:12,016 - pycomo - INFO - Process Pool Logger initialized.
2025-02-11 16:58:12,016 - pycomo - INFO - Utils Logger initialized.
2025-02-11 16:58:12,018 - pycomo - INFO - Multiprocess Logger initialized.
2025-02-11 16:58:12,156 - pycomo - INFO - Log level set to 20
2025-02-11 16:58:12,937 - pycomo - INFO - Logger initialized.
2025-02-11 16:58:12,938 - pycomo - INFO - Process Pool Logger initialized.
2025-02-11 16:58:12,938 - pycomo - INFO - Utils Logger initialized.
2025-02-11 16:58:12,940 - pycomo - INFO - Multiprocess Logger initialized.
2025-02-11 16:58:13,042 - pycomo - INFO - Log level set to 20
2025-02-11 16:58:13,815 - pycomo - INFO - Logger initialized.
2025-02-11 16:58:13,816 - pycomo - INFO - Process Pool Logger initialized.
2025-02-11 16:58:13,816 - pycomo - INFO - Utils Logger initialized.
2025-02-11 16:58:13,818 - py

reaction_id  min_flux  max_flux
0  dv_fraction_reaction  0.074553  0.393590
1  mh_fraction_reaction  0.000000  0.681848
2  mb_fraction_reaction  0.000000  0.925447
3     community_biomass  0.052128  0.052128

### Sensitivity

The ```sensitivity``` parameter describes the amount of decimal places that should be calculated. The default is set to 6, as the cplex solver may return inaccurate results at a higher value.

In [11]:
# sensitivity = 4
C2.max_growth_rate(sensitivity=4)

2025-02-11 16:58:19,340 - pycomo - INFO - New round: lb: 0.0, ub: 1000.0, x: 0.0002
2025-02-11 16:58:20,217 - pycomo - INFO - Logger initialized.
2025-02-11 16:58:20,218 - pycomo - INFO - Process Pool Logger initialized.
2025-02-11 16:58:20,218 - pycomo - INFO - Utils Logger initialized.
2025-02-11 16:58:20,219 - pycomo - INFO - Multiprocess Logger initialized.
2025-02-11 16:58:20,319 - pycomo - INFO - Log level set to 20
2025-02-11 16:58:21,085 - pycomo - INFO - Logger initialized.
2025-02-11 16:58:21,086 - pycomo - INFO - Process Pool Logger initialized.
2025-02-11 16:58:21,086 - pycomo - INFO - Utils Logger initialized.
2025-02-11 16:58:21,088 - pycomo - INFO - Multiprocess Logger initialized.
2025-02-11 16:58:21,193 - pycomo - INFO - Log level set to 20
2025-02-11 16:58:21,990 - pycomo - INFO - Logger initialized.
2025-02-11 16:58:21,991 - pycomo - INFO - Process Pool Logger initialized.
2025-02-11 16:58:21,991 - pycomo - INFO - Utils Logger initialized.
2025-02-11 16:58:21,993 - p

np.float64(0.0521)

In [12]:
# sensitivity = 2
C2.max_growth_rate(sensitivity=2)

2025-02-11 16:58:22,229 - pycomo - INFO - New round: lb: 0.0, ub: 1000.0, x: 0.02
2025-02-11 16:58:23,158 - pycomo - INFO - Logger initialized.
2025-02-11 16:58:23,159 - pycomo - INFO - Process Pool Logger initialized.
2025-02-11 16:58:23,159 - pycomo - INFO - Utils Logger initialized.
2025-02-11 16:58:23,161 - pycomo - INFO - Multiprocess Logger initialized.
2025-02-11 16:58:23,260 - pycomo - INFO - Log level set to 20
2025-02-11 16:58:24,086 - pycomo - INFO - Logger initialized.
2025-02-11 16:58:24,087 - pycomo - INFO - Process Pool Logger initialized.
2025-02-11 16:58:24,087 - pycomo - INFO - Utils Logger initialized.
2025-02-11 16:58:24,089 - pycomo - INFO - Multiprocess Logger initialized.
2025-02-11 16:58:24,199 - pycomo - INFO - Log level set to 20
2025-02-11 16:58:24,972 - pycomo - INFO - Logger initialized.
2025-02-11 16:58:24,973 - pycomo - INFO - Process Pool Logger initialized.
2025-02-11 16:58:24,973 - pycomo - INFO - Utils Logger initialized.
2025-02-11 16:58:24,975 - pyc

np.float64(0.05)